In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('C:/Users/Emincan/Desktop/Playground'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

C:/Users/Emincan/Desktop/Playground\CrabAgePrediction.csv
C:/Users/Emincan/Desktop/Playground\deneme.ipynb
C:/Users/Emincan/Desktop/Playground\Perfect_Notebook_EnsembleML.ipynb
C:/Users/Emincan/Desktop/Playground\sample_submission.csv
C:/Users/Emincan/Desktop/Playground\submission.csv
C:/Users/Emincan/Desktop/Playground\submission_mattop.csv
C:/Users/Emincan/Desktop/Playground\submission_mattop_cat_xgb_lgb_hgb.csv
C:/Users/Emincan/Desktop/Playground\test.csv
C:/Users/Emincan/Desktop/Playground\train.csv
C:/Users/Emincan/Desktop/Playground\catboost_info\catboost_training.json
C:/Users/Emincan/Desktop/Playground\catboost_info\learn_error.tsv
C:/Users/Emincan/Desktop/Playground\catboost_info\time_left.tsv
C:/Users/Emincan/Desktop/Playground\catboost_info\learn\events.out.tfevents
C:/Users/Emincan/Desktop/Playground\my_trainings\all_models.ipynb
C:/Users/Emincan/Desktop/Playground\my_trainings\CatBoost.ipynb
C:/Users/Emincan/Desktop/Playground\my_trainings\combine-diffWeight.ipynb
C:/Users

In [6]:
train = pd.read_csv('C:/Users/Emincan/Desktop/Playground/train.csv')
train['data_type'] = 0

original = pd.read_csv('C:/Users/Emincan/Desktop/Playground/CrabAgePrediction.csv')
original['data_type'] = 1

test = pd.read_csv('C:/Users/Emincan/Desktop/Playground/test.csv')
test['data_type'] = 0

submission = pd.read_csv('C:/Users/Emincan/Desktop/Playground/sample_submission.csv')

train.shape, original.shape, test.shape, submission.shape

((74051, 11), (3893, 10), (49368, 10), (49368, 2))

In [7]:
train.sample(3)

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age,data_type
7316,7316,M,1.6750,1.300,0.45,42.127357,17.137273,10.857859,11.410674,10,0
51502,51502,M,1.4375,1.175,0.45,32.190857,14.188925,6.591259,8.830869,10,0
46385,46385,I,0.9500,0.750,0.25,8.519025,3.529513,1.970290,2.267960,7,0


In [8]:
original.sample(3)

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age,data_type
1269,M,1.6250,1.300,0.4375,35.876292,17.434942,7.866986,9.525432,9,1
1227,M,1.2750,1.000,0.3500,19.575330,7.342521,4.280774,6.520385,10,1
3121,F,1.4875,1.175,0.3875,33.381536,15.365429,7.626016,8.788345,9,1


In [9]:
test.sample(3)

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,data_type
22902,96953,M,1.550,1.1625,0.4000,31.397071,14.217274,6.080968,9.071840,0
23013,97064,F,1.225,1.0625,0.4000,18.781544,7.852811,3.798833,5.216308,0
31934,105985,I,0.625,0.4125,0.1375,2.168737,0.864660,0.453592,0.566990,0


In [10]:
full = pd.concat([train.drop('id', axis=1), original])
full.shape

(77944, 10)

In [11]:
full.sample(3)

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age,data_type
19289,F,1.3750,0.9625,0.4125,23.714357,9.412034,5.938220,5.244657,12,0
15605,M,1.3625,1.0250,0.4125,23.799405,9.766403,4.450871,6.236890,10,0
15430,I,1.0500,0.8000,0.2875,9.185238,4.153202,1.885242,2.494756,7,0


In [13]:
enc = OrdinalEncoder()

for df in [full, test]:
    df['Sex'] = enc.fit_transform(df['Sex'].values.reshape(-1, 1))
    df['Sex'] = df['Sex'].astype('category')

full.sample(3)

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age,data_type
3813,2.0,1.5000,1.1500,0.4125,30.192218,13.777857,7.470093,9.497082,13,0
21452,0.0,1.5750,1.2125,0.4500,33.027167,19.546980,8.887568,7.668540,9,0
40231,0.0,0.6125,0.5000,0.1625,3.047571,1.247378,0.708738,0.793786,6,0


In [14]:
X = full.drop(['Age'], axis=1)
y = full['Age']

test_X = test.drop('id', axis=1)

X.shape, y.shape, test_X.shape

((77944, 9), (77944,), (49368, 9))

In [16]:
cv = KFold(n_splits=10, shuffle=True, random_state=61)
splits = cv.split(X, y)

In [19]:
%%time
cv_scores_train = []
cv_scores_test = []
train_predictions = []
test_predictions = []
predictions = []

for i, (train_idx, test_idx) in enumerate(splits):
    print("="*30, f"FOLD: {(i+1):2d}", "="*30)
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    lgbm = LGBMRegressor(objective = 'mae', 
                           n_estimators = 5000,
                           max_depth = 10,
                           learning_rate = 0.01,
                           num_leaves = 70,
                           reg_alpha = 3,
                           reg_lambda = 3,
                           subsample = 0.7,
                           colsample_bytree = 0.7).fit(X_train, y_train)
    
    y_pred_train = lgbm.predict(X_train)
    y_pred_test = lgbm.predict(X_test)
    y_pred = lgbm.predict(test_X)
    
    train_predictions.append(y_pred_train)
    test_predictions.append(y_pred_test)
    predictions.append(y_pred)
    
    score_train = mean_absolute_error(y_train, y_pred_train)
    score_test = mean_absolute_error(y_test, y_pred_test)
    
    cv_scores_train.append(score_train)
    cv_scores_test.append(score_test)
    
    print(f"MODELO : {lgbm.__class__.__name__} : TRAIN: {score_train:.5f} TEST: {score_test:.5f}")
    
print("END")

============================== FOLD:  1 ==============================
MODELO : LGBMRegressor : TRAIN: 1.19025 TEST: 1.35839
============================== FOLD:  2 ==============================
MODELO : LGBMRegressor : TRAIN: 1.18145 TEST: 1.39252
============================== FOLD:  3 ==============================
MODELO : LGBMRegressor : TRAIN: 1.19434 TEST: 1.32518
============================== FOLD:  4 ==============================
MODELO : LGBMRegressor : TRAIN: 1.20034 TEST: 1.35807
============================== FOLD:  5 ==============================
MODELO : LGBMRegressor : TRAIN: 1.18724 TEST: 1.34844
============================== FOLD:  6 ==============================
MODELO : LGBMRegressor : TRAIN: 1.18529 TEST: 1.37219
============================== FOLD:  7 ==============================
MODELO : LGBMRegressor : TRAIN: 1.19502 TEST: 1.34538
============================== FOLD:  8 ==============================
MODELO : LGBMRegressor : TRAIN: 1.19148 TEST: 1.34524


In [20]:
# Imprimir las predicciones de cada fold
for fold, (train_preds, test_preds) in enumerate(zip(train_predictions, test_predictions)):
    print(f"Fold {fold+1} - TRAIN: {train_preds[:5]}")
    print(f"Fold {fold+1} - TEST : {test_preds[:5]}")
    print()

Fold 1 - TRAIN: [ 9.89922134  7.95982789  8.95481774 12.12752795  8.61031479]
Fold 1 - TEST : [ 7.87822874 10.86088721  7.02918232 11.09425789 10.32191606]

Fold 2 - TRAIN: [ 9.87715902  7.92524793  9.00219209 11.8917055   8.71559789]
Fold 2 - TEST : [9.91763856 9.96217248 8.05440096 7.98819112 7.85014326]

Fold 3 - TRAIN: [ 9.94403385  7.97219585  8.94211111 11.9826668   8.51817689]
Fold 3 - TEST : [11.82953053 10.90156545  8.73738973 10.53667737  7.9784288 ]

Fold 4 - TRAIN: [ 9.98655484  8.97274228 11.73860252  8.73663061  9.96125304]
Fold 4 - TEST : [ 7.91463898  7.86277834  3.83221323 10.70342017  7.13880577]

Fold 5 - TRAIN: [ 9.87650594  7.90848828  8.99920059 11.76394793  8.43054625]
Fold 5 - TEST : [ 8.60026357  8.71913918  7.94253    10.32715126 10.20350939]

Fold 6 - TRAIN: [ 9.87422416  7.91701728  8.95545007 11.65007932  8.5401323 ]
Fold 6 - TEST : [10.63324774  7.00040654 10.85443964  8.01147603  6.98701776]

Fold 7 - TRAIN: [ 9.91188022  7.93739876  8.48865451  9.9633275

In [21]:
# Calcular el promedio de las predicciones de prueba
predictions_average = np.mean(predictions, axis=0)

# Imprimir el promedio
print("Promedio de las predicciones:")
print(predictions_average[:5])

Promedio de las predicciones:
[ 7.33194781  7.91823614 10.34719021  9.08236917  7.18520679]


In [22]:
submission['Age'] = np.array(np.round(predictions_average, 0), dtype=int)
submission

,id,Age
0,74051,7
1,74052,8
2,74053,10
3,74054,9
4,74055,7
...,...,...
49363,123414,9
49364,123415,8
49365,123416,13
49366,123417,9


In [23]:
submission.to_csv('LGBoostSadece.csv')

In [ ]:
# Done.